## note: this is a bit messy, mostly the code I showed, and it is mostly for the students that need it for their projects

Some more background info about scraping:

https://journocode.com/2018/02/scraping-for-everyone/

https://journocode.com/2019/01/web-scraping-tools-journalists-how-to/

You can find more around, and probably youtube videos are a good idea because there is lots of fiddling. (You are a bit on your own here, because I don't like watching videos about coding)

More background info about what's happening with web pages, servers, and everything:

https://www.altexsoft.com/blog/engineering/web-application-architecture-how-the-web-works/

https://hackr.io/blog/web-application-architecture-definition-models-types-and-more

https://www.peerbits.com/blog/web-application-architecture.html

What's a ReST API: https://en.wikipedia.org/wiki/Representational_state_transfer

And the Document Object Model of web pages: https://en.wikipedia.org/wiki/Document_Object_Model (it is scary, I know)



Sites we tackled, for projects for the students (* ):

https://www.ica.se/recept/
(cheat: look for ashx and asp)

https://www.svenskaspel.se/lotto/resultat/2021-03-03

https://www.architonic.com/en/products/sofas/0/3238218/86

(* ) note to self: it is easier to teach on easy sample pages, instead of going in the messy wild...

APIs to play with:

https://wiki.openstreetmap.org/wiki/API_v0.6

https://www.wikidata.org/wiki/Wikidata:Data_access#MediaWiki_API

https://www.wikidata.org/w/api.php

https://api.nasa.gov/ (sign up for the free account)

https://www.themealdb.com/api.php

https://wiki.openfoodfacts.org/API

https://world.openfoodfacts.org/files/api-documentation.html

http://www.recipepuppy.com/about/api/

https://openweathermap.org/api

More:
https://github.com/public-apis/public-apis#food--drink

In [4]:
# all the tools we need, install them/look them up

import requests
from bs4 import BeautifulSoup as bs
import datetime
from lxml import etree
import lxml.html
import json

In [5]:
# url for the first site, we start from a working result,
lotto_url = "https://www.svenskaspel.se/lotto/resultat/2021-03-03"
# we use requests to access the page
lotto_page = requests.get(lotto_url)

In [6]:
# we check the result
lotto_page

<Response [200]>

In [7]:
# let's have a look at what we got
lotto_page.content[:1000]

b'\n<!DOCTYPE html>\n<html class="os-unknown osver-unknown desktop browser-unknown ft-check-app-version ft-gamelinks-analytics ft-product-start-fullwidth ft-lotto-campaign ft-smarter-header ft-swedbankpay-subscription ft-sportsbook ft-hide-oddset ft-subscription-dialog-eurojackpot ft-subscription-dialog-lotto ft-subscription-dialog-vikinglotto ft-leia-game-catalog ft-top-bar-tickets ft-leia-lordagsgodis ft-andelsspel-online ft-euj-subscription-change-1 ft-in-page-admin-v2 ft-fractiongames-fetch-reserved tenant-tb lotto" lang="sv">\n<head>\n    <meta charset="utf-8">\n    \n    <meta name="description" content="Resultat och r\xc3\xa4tta raden f\xc3\xb6r Lotto L\xc3\xb6rdag och Lotto Onsdag, samt Joker">\n    <meta name="keywords" content="Lotto l\xc3\xb6rdag, Lotto onsdag, joker, r\xc3\xa4tta raden, vinn, jackpot, lottoresultat, Dr\xc3\xb6mvinsten, Miljonregnet, andra chansen">\n    <meta name="robots" content="index, follow">\n    <meta property="og:site_name" content="Svenska Spel">\n

In [8]:
# fiddling with Chrome, we find that for every page we request this from an not documented, not protected API (bad programming)
api_url = "https://api.www.svenskaspel.se/multifetch?urls=/draw/lottosaturday/draws/2113/result|draw/jokersaturday/draws/1904/result&_=1616151084726"
results = requests.get(api_url)
results

<Response [200]>

In [9]:
# let's load it in json and see what's inside
json.loads(results.content)

{'requestId': '11c3f6e6-0951-4955-8422-13e632b61cec',
 'deviceId': 'null',
 'requestInfo': {'dataSource': 'MultiFetch',
  'elapsedTime': 9,
  'apiVersion': 1},
 'responses': [{'result': {'drawResult': [{'drawNumbers': [{'numbers': [34,
         15,
         5,
         22,
         12,
         14,
         18],
        'type': 1,
        'description': 'Regular numbers'},
       {'numbers': [21, 11, 1, 13],
        'type': 2,
        'description': 'Extra numbers'}],
      'type': 1,
      'description': 'Lotto1'},
     {'drawNumbers': [{'numbers': [27, 21, 34, 17, 8, 23, 28],
        'type': 1,
        'description': 'Regular numbers'},
       {'numbers': [26, 25, 2, 1], 'type': 2, 'description': 'Extra numbers'}],
      'type': 2,
      'description': 'Lotto2'}],
    'distribution': [{'type': 1,
      'description': 'Lotto1',
      'distribution': [{'name': '7 rätt', 'winners': 0, 'amount': '0,00'},
       {'name': '6 rätt + tillägg', 'winners': 17, 'amount': '46409,00'},
       {'n

In [10]:
# the url has 2 number we can work with, the one in lottosaturday/draws/NUMBER/result|draw/
# and the ones here draws/NUMBER/result
# we can play with them a bit
# [...]
# in my experiments, only the FIRST one matters

In [11]:
# create a model for the URL, to change the number, try some dates
api_url = "https://api.www.svenskaspel.se/multifetch?urls=/draw/lottosaturday/draws/{}/result|draw/jokersaturday/draws/1904/result&_=1616151084726"
results = []
for i in range(2000, 2015):
    api_url_current = api_url.format(i)
    result = requests.get(api_url_current)
    results.append(json.loads(result.content))

In [12]:
len(results)

15

In [13]:
# OK, we have all of them

In [14]:
# checking one
print(results[5])

{'requestId': '7c05787e-e7e8-4bd2-89db-93eba7a5ff65', 'deviceId': 'null', 'requestInfo': {'dataSource': 'MultiFetch', 'elapsedTime': 45, 'apiVersion': 1}, 'responses': [{'result': {'drawResult': [{'drawNumbers': [{'numbers': [23, 29, 12, 6, 13, 22, 28], 'type': 1, 'description': 'Regular numbers'}, {'numbers': [16, 26, 31, 33], 'type': 2, 'description': 'Extra numbers'}], 'type': 1, 'description': 'Lotto1'}, {'drawNumbers': [{'numbers': [16, 17, 15, 8, 27, 10, 35], 'type': 1, 'description': 'Regular numbers'}, {'numbers': [22, 24, 29, 13], 'type': 2, 'description': 'Extra numbers'}], 'type': 2, 'description': 'Lotto2'}], 'distribution': [{'type': 1, 'description': 'Lotto1', 'distribution': [{'name': '7 rätt', 'winners': 2, 'amount': '1131038,00'}, {'name': '6 rätt + tillägg', 'winners': 16, 'amount': '48473,00'}, {'name': '6 rätt', 'winners': 125, 'amount': '3619,00'}, {'name': '5 rätt', 'winners': 5541, 'amount': '139,00'}, {'name': '4 rätt', 'winners': 88612, 'amount': '24,00'}, {'na

In [15]:
# we have the date from 2019-02-09 and it looks sane

In [16]:
# what if we wanted to play with the URL instead? We can use datetime.date and datetime.timedelta
for i in range(15):
    print(datetime.date.today() - datetime.timedelta(days=i))

2022-03-09
2022-03-08
2022-03-07
2022-03-06
2022-03-05
2022-03-04
2022-03-03
2022-03-02
2022-03-01
2022-02-28
2022-02-27
2022-02-26
2022-02-25
2022-02-24
2022-02-23


In [17]:
# let's give it a try... only Tuesday and Saturday should be there, so if we have 15 dates, we should have 4-5 results...
valid_urls = []
RANGE = 15
for i in range(RANGE):
    extra = datetime.date.today() - datetime.timedelta(days=i)
    lotto_url = "https://www.svenskaspel.se/lotto/resultat/{}".format(extra)
#     print(lotto_url)
    r = requests.get(lotto_url)
    if r.ok:
        valid_urls.append(lotto_url)
# let's check
len(valid_urls)

15

In [18]:
# UH? not good... let's check the results

In [19]:
# this is valid date
lotto = requests.get("https://www.svenskaspel.se/lotto/resultat/2021-03-03")
lotto

<Response [200]>

In [20]:
# is should not be
requests.get("https://www.svenskaspel.se/lotto/resultat/2021-03-02")

<Response [200]>

In [21]:
# damn, they give us the current result, without a 404 result...

Weird choice: https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

Please note my favorite response is **418 I'm a teapot** https://en.wikipedia.org/wiki/Hyper_Text_Coffee_Pot_Control_Protocol

Yes, it is in the protocol. I was serious about programmers and humor. I never joke. (Ehm...)

In [22]:
# so what's inside? Let's use beautifulsoup 
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/
lotto_h = bs(lotto.text, 'html.parser')

In [23]:
# let's try to use a selector
selector = "#correctRowRegion > div > div > div > div > div > div.ball-row-numbers-regular > div:nth-child(2) > div.ball-inner > span"
lotto_h.select(selector)

[]

In [24]:
# that didn't work... let's backtrack and select the script, because they are embedding the results in a javascript

# have I ALREADY complained about the programmers of this website? 'cause I could go on

In [25]:
# we want this script, I showed on Chrome
script_selector = "body > script:nth-child(9)"
lotto_selected = lotto_h.select(script_selector)[0]
str(lotto_selected)[:1000]

'<script type="text/javascript">\n            _svs=window.svs=window.svs||{};\n_core=_svs.core=_svs.core||{};\n_core.analytics=_core.analytics||{};\n_core.analytics.data=_core.analytics.data||{};\n_core.analytics.data.svsanalytics={"config":{"account":"UA-7859264-15","domain":"www.svenskaspel.se","siteSpeedSampleRate":"1","gtmAccount":"GTM-WP3D5R","gtmEnabled":true},"adobeLaunch":{"enabled":true,"script":"","suite":"PROD","site":"tur"},"formFactor":"desktop","extraOptions":{"hitCallbackTimeout":2000},"adobeUser":{"loggedIn":"no"},"user":{"user":{"segment":0,"loggedIn":"no"}}};\n_svs.content=_svs.content||{};\n_svs.content.footer=_svs.content.footer||{};\n_svs.content.footer.data=_svs.content.footer.data||{};\n_svs.content.footer.data.config={"customerService":[{"label":"Vanliga frågor","link":"/kundservice","title":"Vanliga frågor"},{"label":"Telefon: 0770-11 11 11","type":"tel","link":"tel:0770-111111","title":"Telefon"},{"label":"Kontakta oss","link":"/kundservice","title":"Kontakta 

In [26]:
# they have resultData with the data embedded in the javascript file
# I tried splitting on newline, it didn't work, it works with ';'
results = []
for el in lotto_selected.contents[0].split(";"):
    if "resultData" in el:
        print(el)
        break


_svs.lotto.data.resultData={"result":{"drawResult":[{"drawNumbers":[{"numbers":[28,2,31,25,6,23,4],"type":1,"description":"Regular numbers"},{"numbers":[22,9,8,3],"type":2,"description":"Extra numbers"}],"type":1,"description":"Lotto1"},{"drawNumbers":[{"numbers":[26,22,34,33,8,32,31],"type":1,"description":"Regular numbers"},{"numbers":[13,3,14,29],"type":2,"description":"Extra numbers"}],"type":2,"description":"Lotto2"}],"distribution":[{"type":1,"description":"Lotto1","distribution":[{"name":"7 rätt","winners":0,"amount":"0,00"},{"name":"6 rätt + tillägg","winners":9,"amount":"38178,00"},{"name":"6 rätt","winners":67,"amount":"2991,00"},{"name":"5 rätt","winners":2744,"amount":"125,00"},{"name":"4 rätt","winners":41577,"amount":"23,00"},{"name":"Drömvinsten","winners":0,"amount":"0,00"}]},{"type":2,"description":"Lotto2","distribution":[{"name":"7 rätt","winners":0,"amount":"0,00"},{"name":"6 rätt + tillägg","winners":6,"amount":"52934,00"},{"name":"6 rätt","winners":55,"amount":"3

In [27]:
# it looks sane, we have to remove the initial bit to have a json
el.strip('\n_svs.lotto.data.resultData=')

'{"result":{"drawResult":[{"drawNumbers":[{"numbers":[28,2,31,25,6,23,4],"type":1,"description":"Regular numbers"},{"numbers":[22,9,8,3],"type":2,"description":"Extra numbers"}],"type":1,"description":"Lotto1"},{"drawNumbers":[{"numbers":[26,22,34,33,8,32,31],"type":1,"description":"Regular numbers"},{"numbers":[13,3,14,29],"type":2,"description":"Extra numbers"}],"type":2,"description":"Lotto2"}],"distribution":[{"type":1,"description":"Lotto1","distribution":[{"name":"7 rätt","winners":0,"amount":"0,00"},{"name":"6 rätt + tillägg","winners":9,"amount":"38178,00"},{"name":"6 rätt","winners":67,"amount":"2991,00"},{"name":"5 rätt","winners":2744,"amount":"125,00"},{"name":"4 rätt","winners":41577,"amount":"23,00"},{"name":"Drömvinsten","winners":0,"amount":"0,00"}]},{"type":2,"description":"Lotto2","distribution":[{"name":"7 rätt","winners":0,"amount":"0,00"},{"name":"6 rätt + tillägg","winners":6,"amount":"52934,00"},{"name":"6 rätt","winners":55,"amount":"3368,00"},{"name":"5 rätt","

In [28]:
# looks sane, let's try to load it in a dictionary
json.loads(el.strip('\n_svs.lotto.data.resultData='))

{'result': {'drawResult': [{'drawNumbers': [{'numbers': [28,
       2,
       31,
       25,
       6,
       23,
       4],
      'type': 1,
      'description': 'Regular numbers'},
     {'numbers': [22, 9, 8, 3], 'type': 2, 'description': 'Extra numbers'}],
    'type': 1,
    'description': 'Lotto1'},
   {'drawNumbers': [{'numbers': [26, 22, 34, 33, 8, 32, 31],
      'type': 1,
      'description': 'Regular numbers'},
     {'numbers': [13, 3, 14, 29], 'type': 2, 'description': 'Extra numbers'}],
    'type': 2,
    'description': 'Lotto2'}],
  'distribution': [{'type': 1,
    'description': 'Lotto1',
    'distribution': [{'name': '7 rätt', 'winners': 0, 'amount': '0,00'},
     {'name': '6 rätt + tillägg', 'winners': 9, 'amount': '38178,00'},
     {'name': '6 rätt', 'winners': 67, 'amount': '2991,00'},
     {'name': '5 rätt', 'winners': 2744, 'amount': '125,00'},
     {'name': '4 rätt', 'winners': 41577, 'amount': '23,00'},
     {'name': 'Drömvinsten', 'winners': 0, 'amount': '0,00'}]}

In [29]:
# good, let's package it in a function
def _get_lotto_data(date):
    # we format the string from the date
    lotto_url = "https://www.svenskaspel.se/lotto/resultat/{}".format(date)
    r = requests.get(lotto_url)
    lotto_h = bs(lotto.text, 'html.parser')
    # look for the script, THIS is easy to break    
    lotto_selected = lotto_h.select('body > script:nth-child(9)')[0]
    data = None
    for el in lotto_selected.contents[0].split(';'):
        # we use the bit from earlier
        if "resultData" in el:
            data = el
            break
    if data:
        # and SINCE THEY GIVE THE RESULTS even if we give a wrong date, let's check we get the date in the json
        if str(date) in data:
            return json.loads(data.strip('\n_svs.lotto.data.resultData='))
    

In [30]:
# sanity check, we ignore a date w/o draws
assert _get_lotto_data('2021-03-02') is None

In [47]:
sofa_selector = "#product-list-605489412918f > li:nth-child(1) > script"

In [48]:
url =  "https://www.architonic.com/en/products/sofas/0/3238218/86"
page = requests.get(url)
page.text[:1000]

'<!DOCTYPE html>\r\n<!--[if IE 9]>\r\n<html class="no-js lt-ie10" lang="en"\r\n      xmlns="http://www.w3.org/1999/xhtml"> <![endif]-->\r\n<!--[if gt IE 9]><!-->\r\n<html class="no-js" lang="en" xmlns="http://www.w3.org/1999/xhtml">\r\n<!--<![endif]-->\r\n<!--[if !IE]>\r\n<html class="no-js" lang="en" xmlns="http://www.w3.org/1999/xhtml"><![endif]-->\r\n<!-- @todo added fonts-loaded to avoid jumping on fonts, improve fallback  -->\r\n\r\n<head >\r\n  <!--[if lte IE 9]>\r\n  <meta http-equiv="X-UA-Compatible" content="IE=9">\r\n  <![endif]-->\r\n  <meta charset="UTF-8"/>\r\n  <meta name="viewport" content="width=device-width, initial-scale=1.0">\r\n  <meta name="referrer" content="unsafe-url">\r\n  <title>SOFAS - High quality designer SOFAS | Architonic</title>\r\n  <meta name="description" content="Designer SOFAS from international manufacturers ✓ detailled info ✓ high-resolution images ✓ CADs ✓ catalogues ✓ find your nearest dealer."/>\r\n  <meta name="robots" content="noindex,follow"

In [49]:
page_h = bs(page.text, 'html.parser')

In [50]:
selector_model = "#product-list-605489412918f > li:nth-child({}) > script"
for i in range(10):
    selector = selector_model.format(i)
    print(selector)
    print(page_h.select(selector))

#product-list-605489412918f > li:nth-child(0) > script
[]
#product-list-605489412918f > li:nth-child(1) > script
[<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"90 Deegree","image":"https:\/\/image.architonic.com\/img_pro1-6\/156\/1643\/90deegree-produkt-00-sq.jpg","description":"90 Deegree | Sofas | SMV Sitz- &amp; Objektm\u00f6bel","productID":1561643,"url":"https:\/\/www.architonic.com\/en\/product\/smv-sitz-objektmobel-90-deegree\/1561643","brand":{"@type":"Brand","name":"SMV Sitz- &amp; Objektm\u00f6bel"}}          
        </script>]
#product-list-605489412918f > li:nth-child(2) > script
[<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"Blanket sofa","image":"https:\/\/image.architonic.com\/img_pro1-6\/134\/0656\/blanket-dark-grey-2-sq.jpg","description":"Blanket sofa | Sofas | Materia","productID":1340656,"url":"https:\/\/www.architonic.com\/en\/product\/ma

In [51]:
page_h.select(selector)

[<script type="application/ld+json">
             {"@context":"http:\/\/schema.org\/","@type":"Product","name":"Passion","image":"https:\/\/image.architonic.com\/img_pro1-6\/133\/3267\/passion-10512-013-sq.jpg","description":"Passion | Sofas | Raun","productID":1333267,"url":"https:\/\/www.architonic.com\/en\/product\/raun-passion\/1333267","brand":{"@type":"Brand","name":"Raun"}}          
         </script>]

In [52]:
s = "#product-list-center-content > div.center-content-list.row > div > ul"
# s = "#product-list-center-content > div.center-content-list.row"
str(page_h.select(s)[0])[:500]

'<ul class="product-list product-block-grid" data-banner-row="4" data-request-uri="en/action" data-return-url="/en/products/sofas/0/3238218/86" id="product-list-605489412918f">\n<li class="product-overview">\n<!--  Structured data JSON -->\n<script type="application/ld+json">\n            {"@context":"http:\\/\\/schema.org\\/","@type":"Product","name":"90 Deegree","image":"https:\\/\\/image.architonic.com\\/img_pro1-6\\/156\\/1643\\/90deegree-produkt-00-sq.jpg","description":"90 Deegree | Sofas | SMV Sitz- &a'

In [54]:
selected = page_h.select(s)[0]
selected.get('id')

'product-list-605489412918f'

In [55]:
#product-list-60544be1e6a25 > li:nth-child(1) > script
selected = page_h.select(selector)[0]
selected

<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"Passion","image":"https:\/\/image.architonic.com\/img_pro1-6\/133\/3267\/passion-10512-013-sq.jpg","description":"Passion | Sofas | Raun","productID":1333267,"url":"https:\/\/www.architonic.com\/en\/product\/raun-passion\/1333267","brand":{"@type":"Brand","name":"Raun"}}          
        </script>

In [56]:
selected.contents

['\n            {"@context":"http:\\/\\/schema.org\\/","@type":"Product","name":"Passion","image":"https:\\/\\/image.architonic.com\\/img_pro1-6\\/133\\/3267\\/passion-10512-013-sq.jpg","description":"Passion | Sofas | Raun","productID":1333267,"url":"https:\\/\\/www.architonic.com\\/en\\/product\\/raun-passion\\/1333267","brand":{"@type":"Brand","name":"Raun"}}          \n        ']

In [57]:
json.loads(selected.contents[0])

{'@context': 'http://schema.org/',
 '@type': 'Product',
 'name': 'Passion',
 'image': 'https://image.architonic.com/img_pro1-6/133/3267/passion-10512-013-sq.jpg',
 'description': 'Passion | Sofas | Raun',
 'productID': 1333267,
 'url': 'https://www.architonic.com/en/product/raun-passion/1333267',
 'brand': {'@type': 'Brand', 'name': 'Raun'}}

In [99]:
# let's try again to get the results for 15 days:
RANGE = 15
data_dict = {}

for i in range(RANGE):
    date = datetime.date.today() - datetime.timedelta(days=i)
    data = _get_lotto_data(date)
    if data:
        data_dict[str(date)] = data
len(data_dict)

4

In [100]:
# it looks sane

In [101]:
# let's play with sofas now
url =  "https://www.architonic.com/en/products/sofas/0/3238218/86"

page = requests.get(url)
page.text[:1000]

'<!DOCTYPE html>\r\n<!--[if IE 9]>\r\n<html class="no-js lt-ie10" lang="en"\r\n      xmlns="http://www.w3.org/1999/xhtml"> <![endif]-->\r\n<!--[if gt IE 9]><!-->\r\n<html class="no-js" lang="en" xmlns="http://www.w3.org/1999/xhtml">\r\n<!--<![endif]-->\r\n<!--[if !IE]>\r\n<html class="no-js" lang="en" xmlns="http://www.w3.org/1999/xhtml"><![endif]-->\r\n<!-- @todo added fonts-loaded to avoid jumping on fonts, improve fallback  -->\r\n\r\n<head >\r\n  <!--[if lte IE 9]>\r\n  <meta http-equiv="X-UA-Compatible" content="IE=9">\r\n  <![endif]-->\r\n  <meta charset="UTF-8"/>\r\n  <meta name="viewport" content="width=device-width, initial-scale=1.0">\r\n  <meta name="referrer" content="unsafe-url">\r\n  <title>SOFAS - High quality designer SOFAS | Architonic</title>\r\n  <meta name="description" content="Designer SOFAS from international manufacturers ✓ detailled info ✓ high-resolution images ✓ CADs ✓ catalogues ✓ find your nearest dealer."/>\r\n  <meta name="robots" content="noindex,follow"

In [102]:
# more weird stuff, they embed the json in the webpage, as a script...
# #product-list-605737a838aea > li:nth-child(1) > script
page_h = bs(page.text, 'html.parser')
selector = "#product-list-605737a838aea > li:nth-child(1) > script"
len(page_h.find_all("script"))

159

In [109]:
# 159 scripts, I probably don't want to go by hand... let's use the selector
selector_model = "#product-list-605737a838aea > li:nth-child({}) > script"
for i in range(5):
    selector = selector_model.format(i + 1)
    # they start counting from 1... weird for me
    print(selector)
    print(page_h.select(selector))
    
# let's look

#product-list-605737a838aea > li:nth-child(1) > script
[<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"Giorgio Sofa-bed","image":"https:\/\/image.architonic.com\/img_pro1-6\/108\/1121\/giorgio-1-02-sq.jpg","description":"Giorgio Sofa-bed | Sofas | die Collection","productID":1081121,"url":"https:\/\/www.architonic.com\/en\/product\/die-collection-giorgio-sofa-bed\/1081121","brand":{"@type":"Brand","name":"die Collection"}}          
        </script>]
#product-list-605737a838aea > li:nth-child(2) > script
[<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"Finsbury 3 Seater Sofa","image":"https:\/\/image.architonic.com\/img_pro1-6\/115\/8604\/finsbury-3-seat-sofa-02-sq.jpg","description":"Finsbury 3 Seater Sofa | Sofas | Assemblyroom","productID":1158604,"url":"https:\/\/www.architonic.com\/en\/product\/assemblyroom-finsbury-3-seater-sofa\/1158604","brand":{"@type":

When I run it, what I see is:

```
#product-list-605737a838aea > li:nth-child(1) > script
[<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"Giorgio Sofa-bed","image":"https:\/\/image.architonic.com\/img_pro1-6\/108\/1121\/giorgio-1-02-sq.jpg","description":"Giorgio Sofa-bed | Sofas | die Collection","productID":1081121,"url":"https:\/\/www.architonic.com\/en\/product\/die-collection-giorgio-sofa-bed\/1081121","brand":{"@type":"Brand","name":"die Collection"}}          
        </script>]
#product-list-605737a838aea > li:nth-child(2) > script
[<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"Finsbury 3 Seater Sofa","image":"https:\/\/image.architonic.com\/img_pro1-6\/115\/8604\/finsbury-3-seat-sofa-02-sq.jpg","description":"Finsbury 3 Seater Sofa | Sofas | Assemblyroom","productID":1158604,"url":"https:\/\/www.architonic.com\/en\/product\/assemblyroom-finsbury-3-seater-sofa\/1158604","brand":{"@type":"Brand","name":"Assemblyroom"}}          
        </script>]
#product-list-605737a838aea > li:nth-child(3) > script
[<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"DV350-COLIBRi\u0300_FAMILY REFUGE","image":"https:\/\/image.architonic.com\/pro1-3\/20117527\/dv350-colibri-family-refuge-set-01-01-pro-sq-arcit18.jpg","description":"DV350-COLIBRi\u0300_FAMILY REFUGE | Sofas | DVO","productID":20117527,"url":"https:\/\/www.architonic.com\/en\/product\/dvo-dv350-colibri-family-refuge\/20117527","brand":{"@type":"Brand","name":"DVO"}}          
        </script>]
#product-list-605737a838aea > li:nth-child(4) > script
[<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"Happy sofa","image":"https:\/\/image.architonic.com\/img_pro1-6\/113\/8375\/Happy-2-sq.jpg","description":"Happy sofa | Sofas | Swedese","productID":1138375,"url":"https:\/\/www.architonic.com\/en\/product\/swedese-happy-sofa\/1138375","brand":{"@type":"Brand","name":"Swedese"}}          
        </script>]
#product-list-605737a838aea > li:nth-child(5) > script
[<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"Infinity back end module left","image":"https:\/\/image.architonic.com\/img_pro1-6\/125\/0814\/infinity-xl-corner-left-sq.jpg","description":"Infinity back end module left | Sofas | Ethimo","productID":1250814,"url":"https:\/\/www.architonic.com\/en\/product\/ethimo-infinity-back-end-module-left\/1250814","brand":{"@type":"Brand","name":"Ethimo"}}          
        </script>]
```
buf it YOU run it now, it will look like:

```
#product-list-60508dc2d3ec8 > li:nth-child(0) > script
[]
#product-list-60508dc2d3ec8 > li:nth-child(1) > script
[]
#product-list-60508dc2d3ec8 > li:nth-child(2) > script
[]
#product-list-60508dc2d3ec8 > li:nth-child(3) > script
[]
#product-list-60508dc2d3ec8 > li:nth-child(4) > script
[]
#product-list-60508dc2d3ec8 > li:nth-child(5) > script
[]
#product-list-60508dc2d3ec8 > li:nth-child(6) > script
[]
#product-list-60508dc2d3ec8 > li:nth-child(7) > script
[]
#product-list-60508dc2d3ec8 > li:nth-child(8) > script
[]
#product-list-60508dc2d3ec8 > li:nth-child(9) > script
[]
```

Because...


In [110]:
# it looks sane...
# with a problem: it would NOT work in another day!
# in the selector
# #product-list-605737a838aea > li:nth-child(1) > script
# the part after product list CHANGES EVERY DAY
# I guess it is a (weak) protection against scraping... let's break it

In [111]:
# let's find a bit without that generated ID, and extract it
s = "#product-list-center-content > div.center-content-list.row > div > ul"
str(page_h.select(s)[0])[:500]

'<ul class="product-list product-block-grid" data-banner-row="4" data-request-uri="en/action" data-return-url="/en/products/sofas/0/3238218/86" id="product-list-605737a838aea">\n<li class="product-overview">\n<!--  Structured data JSON -->\n<script type="application/ld+json">\n            {"@context":"http:\\/\\/schema.org\\/","@type":"Product","name":"Giorgio Sofa-bed","image":"https:\\/\\/image.architonic.com\\/img_pro1-6\\/108\\/1121\\/giorgio-1-02-sq.jpg","description":"Giorgio Sofa-bed | Sofas | die Coll'

In [114]:
selected = page_h.select(s)[0]
str(selected)[:1000]

'<ul class="product-list product-block-grid" data-banner-row="4" data-request-uri="en/action" data-return-url="/en/products/sofas/0/3238218/86" id="product-list-605737a838aea">\n<li class="product-overview">\n<!--  Structured data JSON -->\n<script type="application/ld+json">\n            {"@context":"http:\\/\\/schema.org\\/","@type":"Product","name":"Giorgio Sofa-bed","image":"https:\\/\\/image.architonic.com\\/img_pro1-6\\/108\\/1121\\/giorgio-1-02-sq.jpg","description":"Giorgio Sofa-bed | Sofas | die Collection","productID":1081121,"url":"https:\\/\\/www.architonic.com\\/en\\/product\\/die-collection-giorgio-sofa-bed\\/1081121","brand":{"@type":"Brand","name":"die Collection"}}          \n        </script>\n<div class="product-overview-container">\n<a class="hidden-title-link prediction-item" data-item-id="1081121" data-item-type="product" data-log-type="log_click_object" data-navi-area-id="products" data-navi-group-id="3238218" href="https://www.architonic.com/en/product/die-colle

In [115]:
# neat, what we want is in the ID
selected.get('id')

'product-list-605737a838aea'

In [116]:
# let's build our selector
selector = "#{} > li:nth-child(1) > script".format(selected.get('id'))
selector

'#product-list-605737a838aea > li:nth-child(1) > script'

In [117]:
# let's try again
selected = page_h.select(selector)[0]
selected

<script type="application/ld+json">
            {"@context":"http:\/\/schema.org\/","@type":"Product","name":"Giorgio Sofa-bed","image":"https:\/\/image.architonic.com\/img_pro1-6\/108\/1121\/giorgio-1-02-sq.jpg","description":"Giorgio Sofa-bed | Sofas | die Collection","productID":1081121,"url":"https:\/\/www.architonic.com\/en\/product\/die-collection-giorgio-sofa-bed\/1081121","brand":{"@type":"Brand","name":"die Collection"}}          
        </script>

In [118]:
json.loads(selected.contents[0])

{'@context': 'http://schema.org/',
 '@type': 'Product',
 'name': 'Giorgio Sofa-bed',
 'image': 'https://image.architonic.com/img_pro1-6/108/1121/giorgio-1-02-sq.jpg',
 'description': 'Giorgio Sofa-bed | Sofas | die Collection',
 'productID': 1081121,
 'url': 'https://www.architonic.com/en/product/die-collection-giorgio-sofa-bed/1081121',
 'brand': {'@type': 'Brand', 'name': 'die Collection'}}

In [119]:
# and now we could package it in a function, and a good loop, going throgh all the pages, and all the sofas

In [157]:
# ICA for last
# first of all, I found a weird API call that gives me the full list of recipes (!)
url =  "https://www.ica.se/Templates/ajaxresponse.aspx?ajaxFunction=RecipeListMdsa&mdsarowentityid=&num=16&query=pasta&sortbymetadata=Climate&id=12&_hour=17"
page = requests.get(url)
page.text[:1000]

'\ufeff\r\n\r\n        <script type="application/ld+json">\r\n          {"@context":"http://schema.org","@type":"ItemList","itemListElement":[{"@type":"ListItem","url":"https://www.ica.se/recept/lime-och-ingefarsmarinerad-blomkal-och-linssallad-719671/","position":1},{"@type":"ListItem","url":"https://www.ica.se/recept/tahini-592915/","position":2},{"@type":"ListItem","url":"https://www.ica.se/recept/kryddig-tomatsas-717931/","position":3},{"@type":"ListItem","url":"https://www.ica.se/recept/mole-paste-mexikansk-kryddblandning-722752/","position":4},{"@type":"ListItem","url":"https://www.ica.se/recept/tomatsas-grundrecept-722161/","position":5},{"@type":"ListItem","url":"https://www.ica.se/recept/guldmjolk-721832/","position":6},{"@type":"ListItem","url":"https://www.ica.se/recept/kramig-risoni-med-spenat-mynta-och-arter-722470/","position":7},{"@type":"ListItem","url":"https://www.ica.se/recept/hemmagjord-gnocchi-727225/","position":8},{"@type":"ListItem","url":"https://www.ica.se/rec

In [155]:
# in the URL, we have query for the ingrediends, and num... that is how many results
ica_b = bs(page.text, 'html.parser')
# again, we get a json... but apparently it gives MAX 92 results

In [174]:
# we can use start=X to get around it
# this is the complete url, fragile
url =  "https://www.ica.se/Templates/ajaxresponse.aspx?ajaxFunction=RecipeListMdsa&mdsarowentityid=&start={}&num={}&query=pasta&sortbymetadata=Climate&id=12&_hour=17"
print(url.format(0,10))
page = requests.get(url.format(0,10))
page.text[:1000]

https://www.ica.se/Templates/ajaxresponse.aspx?ajaxFunction=RecipeListMdsa&mdsarowentityid=&start=0&num=10&query=pasta&sortbymetadata=Climate&id=12&_hour=17


'\ufeff\r\n\r\n        <script type="application/ld+json">\r\n          {"@context":"http://schema.org","@type":"ItemList","itemListElement":[{"@type":"ListItem","url":"https://www.ica.se/recept/lime-och-ingefarsmarinerad-blomkal-och-linssallad-719671/","position":1},{"@type":"ListItem","url":"https://www.ica.se/recept/tahini-592915/","position":2},{"@type":"ListItem","url":"https://www.ica.se/recept/kryddig-tomatsas-717931/","position":3},{"@type":"ListItem","url":"https://www.ica.se/recept/mole-paste-mexikansk-kryddblandning-722752/","position":4},{"@type":"ListItem","url":"https://www.ica.se/recept/tomatsas-grundrecept-722161/","position":5},{"@type":"ListItem","url":"https://www.ica.se/recept/guldmjolk-721832/","position":6},{"@type":"ListItem","url":"https://www.ica.se/recept/kramig-risoni-med-spenat-mynta-och-arter-722470/","position":7},{"@type":"ListItem","url":"https://www.ica.se/recept/hemmagjord-gnocchi-727225/","position":8},{"@type":"ListItem","url":"https://www.ica.se/rec

In [179]:
# so I can get as many as I want with:
for i in range(5):
    current_url = url.format(i*10, 10)
    print(current_url)
    page = requests.get(current_url)
    print(page.text[:1000])

https://www.ica.se/Templates/ajaxresponse.aspx?ajaxFunction=RecipeListMdsa&mdsarowentityid=&start=0&num=10&query=pasta&sortbymetadata=Climate&id=12&_hour=17
﻿

        <script type="application/ld+json">
          {"@context":"http://schema.org","@type":"ItemList","itemListElement":[{"@type":"ListItem","url":"https://www.ica.se/recept/lime-och-ingefarsmarinerad-blomkal-och-linssallad-719671/","position":1},{"@type":"ListItem","url":"https://www.ica.se/recept/tahini-592915/","position":2},{"@type":"ListItem","url":"https://www.ica.se/recept/kryddig-tomatsas-717931/","position":3},{"@type":"ListItem","url":"https://www.ica.se/recept/mole-paste-mexikansk-kryddblandning-722752/","position":4},{"@type":"ListItem","url":"https://www.ica.se/recept/tomatsas-grundrecept-722161/","position":5},{"@type":"ListItem","url":"https://www.ica.se/recept/guldmjolk-721832/","position":6},{"@type":"ListItem","url":"https://www.ica.se/recept/kramig-risoni-med-spenat-mynta-och-arter-722470/","position":7},{"

In [180]:
# sigh, it looks like the following pages do not have the json... we would need to act differently

In [181]:
# let's try to select it
ica_selector = "head > script"
ica_b.select(ica_selector)

[]

In [182]:
# uh... it doesn't work...
# (it is NOT a well formed HTML and beautiful soup does not like it)
# long story short (insert severa hours of trying) we can use XPATH and lxml

In [186]:
# we can use start=X to get around it
# this is the complete url, fragile
url =  "https://www.ica.se/Templates/ajaxresponse.aspx?ajaxFunction=RecipeListMdsa&mdsarowentityid=&start={}&num={}&query=pasta&sortbymetadata=Climate&id=12&_hour=17"
page = requests.get(url.format(0,10))
# there is something in the body... interesting?
ica_x = lxml.html.fromstring(page.content)
t = ica_x.xpath('/html/body')[0]
str(t.text_content())[:1000]

'\r\n    \r\n    \r\n    \r\n        var facet = $.parseJSON(\'{"1-lov":136,"2-lov":593,"3-lov":266,"abborre":1,"advent":2,"alla-hjartans-dag":14,"alpro-v1-4-2021":2,"amerikansk":10,"ananas":6,"anka":2,"apelsin":27,"arkiv":29,"arla-2017":2,"arla-v2-4-2021":1,"asiatisk":32,"aubergine":49,"avancerad":23,"avokado":30,"bacon":129,"baka":1,"bakad":3,"bakverk":1,"banan":2,"barilla-2017":6,"barilla-2018":5,"barilla-v35-36":9,"barn":430,"barnmat":6,"basilika":221,"biff":7,"biffar":23,"billiga-veckan":574,"blomkal":27,"blackfisk":3,"blamusslor":13,"bolognese":16,"bratwurst":1,"broccoli":114,"brunch":5,"brysselkal":6,"brollop":6,"brost":8,"buffe":54,"bulgur":2,"bullar":1,"burgare":1,"bar":3,"bonor":148,"caesarsallad":1,"cannelloni":16,"carbonara":26,"cashew":2,"champinjon":104,"chili":119,"chiligryta":1,"chipotle":2,"choklad":2,"chorizo":36,"chevre":4,"citron":201,"cottage-cheese":9,"couscous":2,"creme-fraiche":226,"curry":39,"dagens-enkla":29,"dessert":3,"det-lonar-sig-2015":1,"dill":51,"dinkel

In [187]:
# some python splitting magic
t.text_content().split('{')[1].split('}')[0]

'"1-lov":136,"2-lov":593,"3-lov":266,"abborre":1,"advent":2,"alla-hjartans-dag":14,"alpro-v1-4-2021":2,"amerikansk":10,"ananas":6,"anka":2,"apelsin":27,"arkiv":29,"arla-2017":2,"arla-v2-4-2021":1,"asiatisk":32,"aubergine":49,"avancerad":23,"avokado":30,"bacon":129,"baka":1,"bakad":3,"bakverk":1,"banan":2,"barilla-2017":6,"barilla-2018":5,"barilla-v35-36":9,"barn":430,"barnmat":6,"basilika":221,"biff":7,"biffar":23,"billiga-veckan":574,"blomkal":27,"blackfisk":3,"blamusslor":13,"bolognese":16,"bratwurst":1,"broccoli":114,"brunch":5,"brysselkal":6,"brollop":6,"brost":8,"buffe":54,"bulgur":2,"bullar":1,"burgare":1,"bar":3,"bonor":148,"caesarsallad":1,"cannelloni":16,"carbonara":26,"cashew":2,"champinjon":104,"chili":119,"chiligryta":1,"chipotle":2,"choklad":2,"chorizo":36,"chevre":4,"citron":201,"cottage-cheese":9,"couscous":2,"creme-fraiche":226,"curry":39,"dagens-enkla":29,"dessert":3,"det-lonar-sig-2015":1,"dill":51,"dinkel":1,"dop":11,"dragon":9,"dressing":1,"dryck":2,"edamamebonor":1

In [188]:
# let's try
json.loads('{' + t.text_content().split('{')[1].split('}')[0] + '}')

{'1-lov': 136,
 '2-lov': 593,
 '3-lov': 266,
 'abborre': 1,
 'advent': 2,
 'alla-hjartans-dag': 14,
 'alpro-v1-4-2021': 2,
 'amerikansk': 10,
 'ananas': 6,
 'anka': 2,
 'apelsin': 27,
 'arkiv': 29,
 'arla-2017': 2,
 'arla-v2-4-2021': 1,
 'asiatisk': 32,
 'aubergine': 49,
 'avancerad': 23,
 'avokado': 30,
 'bacon': 129,
 'baka': 1,
 'bakad': 3,
 'bakverk': 1,
 'banan': 2,
 'barilla-2017': 6,
 'barilla-2018': 5,
 'barilla-v35-36': 9,
 'barn': 430,
 'barnmat': 6,
 'basilika': 221,
 'biff': 7,
 'biffar': 23,
 'billiga-veckan': 574,
 'blomkal': 27,
 'blackfisk': 3,
 'blamusslor': 13,
 'bolognese': 16,
 'bratwurst': 1,
 'broccoli': 114,
 'brunch': 5,
 'brysselkal': 6,
 'brollop': 6,
 'brost': 8,
 'buffe': 54,
 'bulgur': 2,
 'bullar': 1,
 'burgare': 1,
 'bar': 3,
 'bonor': 148,
 'caesarsallad': 1,
 'cannelloni': 16,
 'carbonara': 26,
 'cashew': 2,
 'champinjon': 104,
 'chili': 119,
 'chiligryta': 1,
 'chipotle': 2,
 'choklad': 2,
 'chorizo': 36,
 'chevre': 4,
 'citron': 201,
 'cottage-cheese'

In [189]:
# weird, no idea what's that, let's try again
xpath = '/html/head/script'
t = ica_x.xpath(xpath)[0]
t.text_content()

'\r\n          {"@context":"http://schema.org","@type":"ItemList","itemListElement":[{"@type":"ListItem","url":"https://www.ica.se/recept/lime-och-ingefarsmarinerad-blomkal-och-linssallad-719671/","position":1},{"@type":"ListItem","url":"https://www.ica.se/recept/tahini-592915/","position":2},{"@type":"ListItem","url":"https://www.ica.se/recept/kryddig-tomatsas-717931/","position":3},{"@type":"ListItem","url":"https://www.ica.se/recept/mole-paste-mexikansk-kryddblandning-722752/","position":4},{"@type":"ListItem","url":"https://www.ica.se/recept/tomatsas-grundrecept-722161/","position":5},{"@type":"ListItem","url":"https://www.ica.se/recept/guldmjolk-721832/","position":6},{"@type":"ListItem","url":"https://www.ica.se/recept/kramig-risoni-med-spenat-mynta-och-arter-722470/","position":7},{"@type":"ListItem","url":"https://www.ica.se/recept/hemmagjord-gnocchi-727225/","position":8},{"@type":"ListItem","url":"https://www.ica.se/recept/pasta-pesto-med-pinjenotter-och-farsk-basilika-722577

In [190]:
json.loads(str(t.text_content()).strip())

{'@context': 'http://schema.org',
 '@type': 'ItemList',
 'itemListElement': [{'@type': 'ListItem',
   'url': 'https://www.ica.se/recept/lime-och-ingefarsmarinerad-blomkal-och-linssallad-719671/',
   'position': 1},
  {'@type': 'ListItem',
   'url': 'https://www.ica.se/recept/tahini-592915/',
   'position': 2},
  {'@type': 'ListItem',
   'url': 'https://www.ica.se/recept/kryddig-tomatsas-717931/',
   'position': 3},
  {'@type': 'ListItem',
   'url': 'https://www.ica.se/recept/mole-paste-mexikansk-kryddblandning-722752/',
   'position': 4},
  {'@type': 'ListItem',
   'url': 'https://www.ica.se/recept/tomatsas-grundrecept-722161/',
   'position': 5},
  {'@type': 'ListItem',
   'url': 'https://www.ica.se/recept/guldmjolk-721832/',
   'position': 6},
  {'@type': 'ListItem',
   'url': 'https://www.ica.se/recept/kramig-risoni-med-spenat-mynta-och-arter-722470/',
   'position': 7},
  {'@type': 'ListItem',
   'url': 'https://www.ica.se/recept/hemmagjord-gnocchi-727225/',
   'position': 8},
  {'

In [169]:
# bingo
len(json.loads(str(t.text_content()).strip())['itemListElement'])

16